# Hyperparameter tuning and cross-validation

Test the data on various models using ensemble learning. Determine the best model for use.

---

### 1. Load packages and open dataset

This example will demonstrate on the middle point which we will use to determine which model is best for predicting all three points.

In [1]:
import pandas as pd
import numpy as np
import xarray as xr
import plotly.express as px
from pycaret.regression import *

In [2]:
df = pd.read_csv('../data/clean/Middle_velocity_series_clean.csv')

In [3]:
df.head()

,time,VelocitySeries
0,2015-01-30 12:00:00,779.53534
1,2015-02-11 12:00:00,738.42413
2,2015-07-29 12:00:00,741.93330
3,2015-09-03 12:00:00,736.29333
4,2015-09-15 12:00:00,731.44696


Create a monthly rolling average of the data to plot with the velocity data.

In [4]:
# create 12 month moving average
df.rename(columns={'VelocitySeries': 'velocity'}, inplace=True)
df['MA12'] = df['velocity'].rolling(12).mean()

# plot the data and monthly average
fig = px.line(df, x="time", y=["velocity", "MA12"], template = 'plotly_dark')
fig.show()

Create a series to add to the dataframe for interpretation of time series data. Drop the values that are unnecessary, leaving series, time, and velocity values.

In [5]:
# create a sequence of numbers
df['series'] = np.arange(1,len(df)+1)
df['time'] = pd.to_datetime(df['time'])

# drop unnecessary columns and re-arrange
df = df[['series', 'time', 'velocity']] 

# check the head of the dataset
df.head()

,series,time,velocity
0,1,2015-01-30 12:00:00,779.53534
1,2,2015-02-11 12:00:00,738.42413
2,3,2015-07-29 12:00:00,741.93330
3,4,2015-09-03 12:00:00,736.29333
4,5,2015-09-15 12:00:00,731.44696


Split the data into train and test data. We will use ~75% for training and ~25% for testing.

In [6]:
# split data into train-test set
train = df[df['time'] < pd.to_datetime('06-01-2021')]
test = df[df['time'] >= pd.to_datetime('06-01-2021')]

# check shape
print("Shape of training data:", train.shape, "Shape of test data:", test.shape)

# calculate percentage
train_percentage = (len(train) / len(df)) * 100
test_percentage = (len(test) / len(df)) * 100

print(f"Training data percentage: {train_percentage:.2f}%")
print(f"Testing data percentage: {test_percentage:.2f}%")

Shape of training data: (272, 3) Shape of test data: (92, 3)
Training data percentage: 74.73%
Testing data percentage: 25.27%


### 3. Initialize the model

Here we will use Pycaret's setup function to initialize the model, passing the training data, testing data, and the features to predict. The function outputs information about the pipeline that will be used for model selection and tuning.

In [7]:
# initialize setup
s = setup(data = train, test_data = test, 
          target = 'velocity', fold_strategy = 'timeseries', 
          numeric_features = ['time', 'series'], fold = 6, 
          transform_target = True, session_id = 123,
          data_split_shuffle = False, fold_shuffle = False)

,Description,Value
0,Session id,123
1,Target,velocity
2,Target type,Regression
3,Original data shape,"(364, 3)"
4,Transformed data shape,"(364, 5)"
5,Transformed train set shape,"(272, 5)"
6,Transformed test set shape,"(92, 5)"
7,Numeric features,2
8,Date features,1
9,Preprocess,True


### 4. Compare models

Now, we can compare the models using cross-validation. We sort them based on the mean absolute error (MAE) value which is a measure of error that is not as sensitive to outliers are mean squared error and root mean squared error. We'll also test the first 4 models on the test data, so select those with `n_select`.

In [13]:
# Test the models
comparison = compare_models(n_select=4, sort='MAE', cross_validation=True)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,32.2048,2297.6731,45.5717,-0.5879,0.0554,0.0393,0.0583
omp,Orthogonal Matching Pursuit,36.4665,2539.1248,46.7146,-0.5131,0.0569,0.0455,0.0050
rf,Random Forest Regressor,36.9053,2479.3913,47.6877,-0.7996,0.0583,0.0460,0.0183
lr,Linear Regression,36.9575,2669.0707,47.1474,-0.4471,0.0574,0.0461,0.0183
ridge,Ridge Regression,37.0559,2656.4237,47.2130,-0.4635,0.0575,0.0462,0.0067
lar,Least Angle Regression,37.4024,2689.7239,47.3562,-0.4705,0.0577,0.0466,0.0067
knn,K Neighbors Regressor,37.6565,2456.7909,48.8224,-1.5431,0.0596,0.0462,0.0067
llar,Lasso Least Angle Regression,38.0298,2562.6425,49.0576,-1.4234,0.0597,0.0459,0.0050
lasso,Lasso Regression,38.0298,2562.6425,49.0576,-1.4234,0.0597,0.0459,0.0083
en,Elastic Net,38.2198,2569.9167,49.2070,-1.4464,0.0599,0.0461,0.0067


Light Gradient Boosting Machine (LightGBM) outperformed all other models in almost every category. This could be due to overfitting so let's test it on the test set.

In [15]:
prediction_holdout = predict_model(comparison[0]);

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,38.1687,2370.2136,48.6848,-0.1064,0.0580,0.0447


It performed slightly worse on the test data. Let's test next 3 models and see which performs well on the test data.

In [16]:
# Initialize a dictionary to store the MAE scores
mae_scores = {}

# Iterate through the top 4 models and test them on the test data
for model in comparison:
    # Predict on the test data
    predictions = predict_model(model, data=test)
    
    # Calculate the MAE score
    mae = np.mean(np.abs(predictions['velocity'] - predictions['prediction_label']))
    
    # Store the MAE score in the dictionary
    mae_scores[model] = mae

# Determine the model with the best MAE score
best_model = min(mae_scores, key=mae_scores.get)
best_mae_score = mae_scores[best_model]

print(f"The best model is {best_model} with a MAE score of {best_mae_score:.4f}")

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,38.1687,2370.2136,48.6848,-0.1064,0.0580,0.0447


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Orthogonal Matching Pursuit,35.5600,2468.5898,49.6849,-0.1523,0.0596,0.0430


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Random Forest Regressor,32.8528,2028.5126,45.0390,0.0531,0.0538,0.0389


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Linear Regression,47.9550,4259.5550,65.2653,-0.9883,0.0771,0.0581


The best model is RandomForestRegressor(n_jobs=-1, random_state=123) with a MAE score of 32.8528


Random Forest performed the best on the test data. We will use this moving forward.

### 5. Tune the model

Here we use hyperparameter tuning to optimize our chosen model. We'll optimize it under mean absolute error (MAE).

In [17]:
dt = create_model('rf', fold=6)

tune_model(dt, optimize='MAE')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,46.5757,2955.4998,54.3645,-0.5671,0.0668,0.0592
1,49.6955,3688.1763,60.7304,-0.3372,0.0742,0.0622
2,28.5701,1216.3119,34.8757,-3.0317,0.0415,0.0340
3,40.1318,3876.4787,62.2614,-0.4281,0.0803,0.0533
4,37.5735,2627.5345,51.2595,0.0436,0.0599,0.0444
5,18.8849,512.3467,22.6351,-0.4771,0.0270,0.0227
Mean,36.9053,2479.3913,47.6877,-0.7996,0.0583,0.0460
Std,10.5053,1233.3068,14.3273,1.0168,0.0186,0.0140


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,32.2257,2250.6570,47.4411,-0.1934,0.0570,0.0388
1,54.2872,4542.5424,67.3984,-0.6470,0.0811,0.0639
2,46.0154,2358.7261,48.5667,-6.8185,0.0590,0.0544
3,35.6285,2746.2370,52.4046,-0.0117,0.0678,0.0458
4,35.5792,2870.6988,53.5789,-0.0450,0.0619,0.0407
5,13.8449,328.1206,18.1141,0.0540,0.0218,0.0166
Mean,36.2635,2516.1636,47.9173,-1.2769,0.0581,0.0434
Std,12.5148,1236.0996,14.8357,2.4890,0.0180,0.0147


Fitting 6 folds for each of 10 candidates, totalling 60 fits


RandomForestRegressor(bootstrap=False, max_depth=11, max_features='sqrt',
                      min_impurity_decrease=0.0002, min_samples_leaf=5,
                      min_samples_split=7, n_estimators=200, n_jobs=-1,
                      random_state=123)

The tuned random forest performed very slightly better than the original. We'll use these parameters moving forward. 